# Analiza dnevnih migracij v Sloveniji

V nalogi bom analiziral migracije med Slovenskimi občinami in ugotavljal v kakšnem razmerju so cene nepremičnin ter  delovne razmere in ugotavljal kje bi bilo mogoče izboljšat zaslužek in standard življenja.

Na začetku naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [43]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

pot_do_podatkov = "./podatki"

nepremicnine = pd.read_csv(os.path.join(pot_do_podatkov, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(pot_do_podatkov, "podjetja.csv"))
razdalje = pd.read_csv(os.path.join(pot_do_podatkov, "razdalje.csv"), index_col="zacetek")
migracije_med_obcinami = pd.read_csv(os.path.join(pot_do_podatkov, "migracije.csv"))

obcine = pd.read_csv(os.path.join(pot_do_podatkov, "obcine.csv"), index_col="ime")

Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno. Prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah, ki nam bodo pomagale ocenit koliko povprečno posameznik izgubi časa in denarja zaradi potrebnih poti.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit pri katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini.

In [44]:
nepremicnine.info()
podjetja.info()
migracije_med_obcinami.info()
razdalje.info()
obcine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24934 entries, o6358608 to o6353469
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   posredovanje   24934 non-null  object 
 1   naslov         24934 non-null  object 
 2   url            24934 non-null  object 
 3   kratek_opis    24934 non-null  object 
 4   vrsta          24933 non-null  object 
 5   regija         24933 non-null  object 
 6   upravna_enota  24933 non-null  object 
 7   obcina         24757 non-null  object 
 8   velikost       24925 non-null  float64
 9   cena           24930 non-null  float64
 10  leto           20022 non-null  float64
 11  sobe           5450 non-null   float64
 12  agencija       24934 non-null  object 
dtypes: float64(4), object(9)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9925 entries, 0 to 9924
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------          

## Nepremičnine

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

Cene nepremičnin nam bodo pomagale ocenit, če je razlika v ceni nepremičnine, glede na to kam ljudje iz te občine migrirajo, smiselna glede na čas in stroške poti. V idealni situaciji bi živel tam, kjer je cena nepremičnin majhna, pot kratka, zaslužek pa velik.

In [45]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

## Migracije

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

To bomo izračunali na sledeč način:

1. Najprej bomo izračunali absolutno število ljudi, ki vsak dan migrira v Ljubljano. Ta podatek dobimo kot razliko delavno aktivnih prebivalcev, ki delajo v Ljubljani (po delovnem mestu) in delavno aktivnih prebivalcev, ki živijo v Ljubljani (po prebivališču), pomnožen z deležem lokalnih delavcev.

1. Glede na izbrano občino bomo izračunali kakšna je verjetnost, da gre ta oseba v druge občine. To bomo določili kot relativno število migracije v posamezno občino glede na vse ostale občine razen te, iz katere oseba prihaja. Ker ljudje raje delajo bližje doma, bom vrednosti obtežil glede na oddaljenost od dane občine.


In [46]:
migracije_po_obcinah = migracije_med_obcinami.groupby('obcina').mean()

# Dodaj podatke v občine.
obcine["delez_migrantov"] = migracije_po_obcinah.delez_migrantov
obcine["delez_lokalnih_delavcev"] = migracije_po_obcinah.delez_lokalnih_delavcev

# Koliko ljudi pride vsak dan v to občino iz druih Slovenskih občin.
obcine["st_migrantov_v_obcino"] = obcine.st_delovno_aktivnih_po_delovnem_mestu - (obcine.st_delovno_aktivnih_po_prebivaliscu * (obcine.delez_lokalnih_delavcev / 100))

# Za vsako občino dobimo podatek o tem koliko časa (sekund) in kakšno razdaljo (metrov)
# potrebuje oseba za pot do vseh ostalih občin.
def razdalja_in_cas_iz_obcine(obcina):
    return razdalje[
        (razdalje.index == obcina) &
        # nekateri podatki imajo vrednost 1000km, zato jih filtriramo.
        (razdalje.razdalja < 500_000)
    ].set_index("konec")

# Kakšna je verjetnost, da migrira nekdo iz dane občine v drugo občino.
def doloci_migracije(obcina):
    # Odstranimo občino v kateri smo.
    ostale_obcine = obcine[obcine.index != obcina].st_migrantov_v_obcino
    
    # Verjetnost, da se prebivalec naše občine odpelje v drugo občino.
    verjetnost = ostale_obcine / ostale_obcine.sum()
    
    # Občine otežimo glede na razdaljo od naše občine.
    oddaljenost = razdalja_in_cas_iz_obcine(obcina).razdalja
    utezi = (1 - pow((oddaljenost / oddaljenost.max()), 0.5))
    
    utezena_verjetnost = verjetnost * utezi

    return utezena_verjetnost / utezena_verjetnost.sum()
    
# Uporabimo narejeno funkcijo na vsaki občini, tako da dobimo razpredelnico o tem, 
# kakšna je verjetnost, da gre nekdo iz poljubne občine v poljubno občino.
migracije = pd.Series(
    obcine.index, 
    name="obcine", 
    index=obcine.index
).apply(
    doloci_migracije
).transpose()

# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije_obcina = migracije[obcina]

    df = pd.DataFrame({ 
        "razdalja": razdalja_in_cas_iz_obcine(obcina).razdalja, 
        "cas": razdalja_in_cas_iz_obcine(obcina).cas, 
        "migracije": migracije_obcina
    }, index=migracije_obcina.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
poti = pd.DataFrame(
    pd.Series(obcine.index, index=obcine.index).apply(pot),
    columns=["razdalja", "cas"]
)

# poti.sort_values("razdalja").tail(20)
# poti["Grosuplje"], poti["Maribor"]

# Dodaj k občinam
obcine["razdalja_migracije"] = poti.razdalja
obcine["cas_migracije"] = poti.cas

Na podlagi podatkov o migracijah lahko ocenimo koliko denarja posamezna občina nameni migracijam. To bom naredil tako, da bom na podlagi povprečne dolžine poti do delovnega mesta, izračunal koliko bencina porabijo ljudje za pot ter iz cene naftnih derivatov izračunal oceno potnih stroškov. Na podlagi povprečne neto plače v občini bom izračunal urno postavko delavca ter upošteval še povprečni čas vožnje na delovno mesto.

Povprečno porabo avtomobila sem pobral iz [te](https://www.iea.org/reports/fuel-consumption-of-cars-and-vans) strani in znaša malo več kot 7L/100km, kar bom zaokrožil na 7L/100km. Za ceno 1L goriva bom vzel kar 1€, saj je toliko znašala cena goriva v zadnjem letu.

Izračunal bom še znesek nadomestila, ki ga migrantje prejmejo od svojih podjetji kot obračun prevoza na delo. Povrnjen znesek iz [vira](https://mladipodjetnik.si/novice-in-dogodki/novice/obracun-prevoz-na-delo-ter-kilometrine-2) znaša 0.18€/km.

In [47]:
# Stroški prevoza
poraba_goriva = 7/100
cena_goriva = 1

delovni_dnevi_v_mescu = 4 * 5
st_potovanj = delovni_dnevi_v_mescu * 2

obcine["stroski_goriva"] = (obcine.razdalja_migracije / 1000) * poraba_goriva * cena_goriva * st_potovanj

# Čas potovanja
delovne_ure_v_mescu = delovni_dnevi_v_mescu * 8
obcine["povprecna_urna_postavka"] = obcine.placa_neto / delovne_ure_v_mescu

obcine["stroski_prevoza"] = (obcine.cas_migracije / 3600) * st_potovanj * obcine.povprecna_urna_postavka

# Povračilo potnih stroškov na prebivalca.
obcine["povracilo_stroskov_migracije"] = 0.18/1 * (obcine.razdalja_migracije / 1000) * st_potovanj

# Združimo vse stroške.
obcine["stroski_migracije_na_delovca"] = obcine.stroski_prevoza + obcine.stroski_goriva
obcine["razlika_v_stroskih_na_delovca"] = obcine.stroski_migracije_na_delovca - obcine.povracilo_stroskov_migracije

# Izračunamo koliko denarja občina izgubi zaradi migracij občanov.
obcine["st_migrantov_iz_obcine"] = obcine.st_delovno_aktivnih_po_prebivaliscu * (obcine.delez_migrantov / 100)
obcine["stroski_migracije"] = obcine.stroski_migracije_na_delovca * obcine.st_migrantov_iz_obcine

In [48]:
stroski_migracije = [
    "razdalja_migracije", 
    "stroski_goriva", 
    "cas_migracije", 
    "povprecna_urna_postavka", 
    "stroski_prevoza", 
    "stroski_migracije_na_delovca",
    "razlika_v_stroskih_na_delovca",
    "st_migrantov_iz_obcine",
    "stroski_migracije"
]

obcine.sort_values("stroski_migracije", ascending=False)[stroski_migracije]

,razdalja_migracije,stroski_goriva,cas_migracije,povprecna_urna_postavka,stroski_prevoza,stroski_migracije_na_delovca,razlika_v_stroskih_na_delovca,st_migrantov_iz_obcine,stroski_migracije
ime,,,,,,,,,
Ljubljana,63503.376203,177.809453,3068.187545,8.010375,273.081476,450.890929,-6.333380,17911.78830,8.076263e+06
Maribor,90177.300907,252.496443,3901.451782,6.780125,293.914786,546.411229,-102.865338,9883.54120,5.400478e+06
Koper/Capodistria,125724.144962,352.027606,5018.146812,7.352375,409.947746,761.975352,-143.238492,6604.99950,5.032847e+06
Kranj,58153.160416,162.828849,2825.489721,7.075000,222.114886,384.943736,-33.759019,11404.21305,4.389980e+06
Nova Gorica,123742.620576,346.479338,5092.983352,7.124688,403.176831,749.656169,-141.290699,4934.81680,3.699416e+06
...,...,...,...,...,...,...,...,...,...
Kostel,110569.214155,309.593800,6158.304114,6.306813,431.547437,741.141237,-54.957105,163.57620,1.212331e+05
Solčava,74956.692419,209.878739,5045.630656,6.279375,352.037856,561.916594,22.228409,111.67175,6.275021e+04
Hodoš/Hodos,144519.904371,404.655732,6367.906238,5.204750,368.259555,772.915288,-267.628024,63.13500,4.879801e+04


## Združenje podatkov o nepremičninah in stroških migracije.

